# Broadway League Data Personal Project - Scrub 1 Data Exploration
Cooper Orio; Summer 2025

In [2]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import tree

from sklearn.metrics import accuracy_score, brier_score_loss
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import CalibrationDisplay

In [3]:
# First, the size and format of the data:
bway_df = pd.read_csv('broadway_league_data.csv')
bway_df_test = bway_df.copy()
bway_df.shape

(59537, 12)

In [4]:
bway_df.head(10)

,Unnamed: 0,Week End,Show,Type,Theatre,#Prev,#Perf,Grosses,Grosses\nPrev Week,Attend,Attend\nPrev Week,% Cap
0,0,06/10/1979,A CHORUS LINE,Musical,Shubert,0,8,"$199,466",$,"9,810",NaN,83%
1,1,06/17/1979,A CHORUS LINE,Musical,Shubert,0,8,"$194,223","$199,466","10,770","9,810",91%
2,2,06/24/1979,A CHORUS LINE,Musical,Shubert,0,8,"$201,091","$194,223","11,376","10,770",97%
3,3,07/01/1979,A CHORUS LINE,Musical,Shubert,0,8,"$198,060","$201,091","11,923","11,376",101%
4,4,06/10/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$35,473",$,"3,408",NaN,85%
5,5,06/17/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$34,786","$35,473","3,387","3,408",85%
6,6,06/24/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$31,867","$34,786","3,232","3,387",81%
7,7,07/01/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$32,531","$31,867","3,373","3,232",84%
8,8,06/08/1980,A CHORUS LINE,Musical,Shubert,0,8,"$195,966",$,"11,238",NaN,95%
9,9,06/08/1980,A DAY IN HOLLYWOOD/A NIGHT IN THE UKRAINE,Musical,Golden,0,8,"$105,223",$,"6,359",NaN,99%


In [5]:
# First, I notice that the first column is just the number of the row entry, so I can remove that:
bway_df = bway_df.drop('Unnamed: 0', axis=1)

# Furthermore, I notice some \n escape characters in the names of some columns, so I rename the columns
bway_df = bway_df.rename(columns={'Grosses\nPrev Week': 'LW Grosses ($)',
                                      'Grosses Prev Week': 'LW Grosses ($)', 
                                      'Attend\nPrev Week': 'LW Attend',
                                      'Attend Prev Week': 'LW Attend',
                                      'Grosses': 'Grosses ($)'})

bway_df.head(10)

,Week End,Show,Type,Theatre,#Prev,#Perf,Grosses ($),LW Grosses ($),Attend,LW Attend,% Cap
0,06/10/1979,A CHORUS LINE,Musical,Shubert,0,8,"$199,466",$,"9,810",NaN,83%
1,06/17/1979,A CHORUS LINE,Musical,Shubert,0,8,"$194,223","$199,466","10,770","9,810",91%
2,06/24/1979,A CHORUS LINE,Musical,Shubert,0,8,"$201,091","$194,223","11,376","10,770",97%
3,07/01/1979,A CHORUS LINE,Musical,Shubert,0,8,"$198,060","$201,091","11,923","11,376",101%
4,06/10/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$35,473",$,"3,408",NaN,85%
5,06/17/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$34,786","$35,473","3,387","3,408",85%
6,06/24/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$31,867","$34,786","3,232","3,387",81%
7,07/01/1979,OH! CALCUTTA!,Musical,Edison,0,8,"$32,531","$31,867","3,373","3,232",84%
8,06/08/1980,A CHORUS LINE,Musical,Shubert,0,8,"$195,966",$,"11,238",NaN,95%
9,06/08/1980,A DAY IN HOLLYWOOD/A NIGHT IN THE UKRAINE,Musical,Golden,0,8,"$105,223",$,"6,359",NaN,99%


In [6]:
# Next, I notice that when any of the financial values are meant to report 0 or no data, they just
# show a dollar sign alone. Similarly, for Attendance fields, they just show NaN. 

# By inference, if the show just opened that week, then the attendance and gross of 
# the previous week especially must be 0, so I decide to impute these fields as such 
# (as strings, to keep type consistency in columns):
bway_df['Grosses ($)'] = bway_df['Grosses ($)'].replace('$', '0', regex=False)
bway_df['LW Grosses ($)'] = bway_df['LW Grosses ($)'].replace('$', '0', regex=False)
bway_df['Attend'] = bway_df['Attend'].fillna('0')
bway_df['LW Attend'] = bway_df['LW Attend'].fillna('0')

In [7]:
# Then, I notice that my columns have less than ideal formats. I want them in their 
# most raw form - datetime, numerics, and strings (when necessary, like with titles and names)
type(bway_df['Week End'][0])

str

In [8]:
# So I reformat my Week End column to be in datetime, rather than a string with the dates.
bway_df['Week End'] = pd.to_datetime(bway_df['Week End'])

# I remove the commas and dollar signs from my monitary fields to make them floats:
bway_df['Grosses ($)'] = bway_df['Grosses ($)'].str.replace('$', '').str.replace(',', '').astype(float)
bway_df['LW Grosses ($)'] = bway_df['LW Grosses ($)'].str.replace('$', '').str.replace(',', '').astype(float)

# I remove the commas from the attendance fields for the same purpose:
bway_df['Attend'] = bway_df['Attend'].str.replace(',', '').astype(float)
bway_df['LW Attend'] = bway_df['LW Attend'].str.replace(',', '').astype(float)

# And I remove the percent signs from the percent capacity field:
bway_df['% Cap'] = bway_df['% Cap'].str.replace('%', '').astype(float)

bway_df['#Prev'] = bway_df['#Prev'].astype(float)
bway_df['#Perf'] = bway_df['#Perf'].astype(float)

In [9]:
bway_df.tail(20)

,Week End,Show,Type,Theatre,#Prev,#Perf,Grosses ($),LW Grosses ($),Attend,LW Attend,% Cap
59517,2025-05-25,MJ,Musical,Neil Simon,0.0,8.0,1241900.0,1142907.0,9760.0,9553.0,88.0
59518,2025-05-25,MOULIN ROUGE! THE MUSICAL,Musical,Al Hirschfeld,0.0,8.0,1277854.0,1152473.0,10235.0,10122.0,98.0
59519,2025-05-25,"OH, MARY!",Play,Lyceum,0.0,8.0,1188742.0,1104396.0,7160.0,7160.0,100.0
59520,2025-05-25,OPERATION MINCEMEAT: A NEW MUSICAL,Musical,Golden,0.0,8.0,800367.0,757916.0,6320.0,6211.0,100.0
59521,2025-05-25,OTHELLO 2025,Play,Ethel Barrymore,0.0,8.0,3327506.0,3206029.0,8344.0,8344.0,100.0
59522,2025-05-25,PIRATES! THE PENZANCE MUSICAL,Musical,Todd Haimes,0.0,8.0,545763.0,496228.0,5693.0,5225.0,98.0
59523,2025-05-25,PURPOSE,Play,Hayes,0.0,8.0,604893.0,558484.0,4647.0,4527.0,100.0
59524,2025-05-25,REAL WOMEN HAVE CURVES: THE MUSICAL,Musical,James Earl Jones,0.0,8.0,405420.0,351281.0,6128.0,5371.0,73.0
59525,2025-05-25,SIX: THE MUSICAL,Musical,Lena Horne,0.0,8.0,712632.0,617336.0,6919.0,5943.0,84.0
59526,2025-05-25,SMASH,Musical,Imperial,0.0,8.0,872706.0,790715.0,9434.0,8858.0,84.0


In [10]:
# Trying it out in one function form:
def BWL_reformat(bway_df):
    '''Informed by First Scrub Exploration, this function
    reformats the pandas dataframe containing Broadway League
    Data into a form that is more useable for analytics and
    Model building. input: pandas df; output: pandas df'''

    # First, I remove the potential leading column that was present
    # when read from the format of a saved pandas CSV:
    if 'Unnamed: 0' in bway_df.columns:
        bway_df = bway_df.drop('Unnamed: 0', axis=1)

    # Next, I rename the previous week columns in a format-robust manner, 
    # using 'LW' to denote 'last week'. Furthermore, sinceI will be 
    # removing the $ from the data, I rename the raw Grosses, too.
    bway_df = bway_df.rename(columns={'Grosses\nPrev Week': 'LW Grosses ($)',
                                      'Grosses Prev Week': 'LW Grosses ($)', 
                                      'Attend\nPrev Week': 'LW Attend',
                                      'Attend Prev Week': 'LW Attend',
                                      'Grosses': 'Grosses ($)'})
    
    # Then, I impute the empty financial & attendance values with 0s:
    bway_df['Grosses ($)'] = bway_df['Grosses ($)'].replace('$', '0', regex=False)
    bway_df['LW Grosses ($)'] = bway_df['LW Grosses ($)'].replace('$', '0', regex=False)
    bway_df['Attend'] = bway_df['Attend'].fillna('0')
    bway_df['LW Attend'] = bway_df['LW Attend'].fillna('0')

    # Finally, I make sure that all of my values are in analyzable formats, with
    # the numerical values as floats (for the sake of consistency), the titles, 
    # theatres, and show types as strings, and the Week Ends as dates.
    bway_df['Week End'] = pd.to_datetime(bway_df['Week End'])

    bway_df['Grosses ($)'] = bway_df['Grosses ($)'].str.replace('$', '').str.replace(',', '').astype(float)
    bway_df['LW Grosses ($)'] = bway_df['LW Grosses ($)'].str.replace('$', '').str.replace(',', '').astype(float)

    bway_df['Attend'] = bway_df['Attend'].str.replace(',', '').astype(float)
    bway_df['LW Attend'] = bway_df['LW Attend'].str.replace(',', '').astype(float)

    bway_df['% Cap'] = bway_df['% Cap'].str.replace('%', '').astype(float)

    bway_df['#Prev'] = bway_df['#Prev'].astype(float)
    bway_df['#Perf'] = bway_df['#Perf'].astype(float)

    return bway_df

In [11]:
testing_function_df = BWL_reformat(bway_df_test)
testing_function_df.head(20)

KeyError: 'LW Grosses ($)'